In [ ]:
# 1-) Yeni Ürünlerin sektörlerini bulmak

import pandas as pd

# CSV dosyalarını yükleyelim
df_2002 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv")
df_2007 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv")
df_2012 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv")
df_2017 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv")
df_2022 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv")

# 2002, 2007, 2012, 2017 ve 2022 yıllarına ait product code'ları set yap
products_2002 = set(df_2002['code'])
products_2007 = set(df_2007['code'])
products_2012 = set(df_2012['code'])
products_2017 = set(df_2017['code'])
products_2022 = set(df_2022['code'])

# 2002, 2012, 2017 ve 2022'de yeni eklenen ürünler
new_products_2007 = products_2007 - products_2002
new_products_2007_df = df_2007[df_2007['code'].isin(new_products_2007)].copy()
new_products_2012 = products_2012 - products_2007
new_products_2012_df = df_2012[df_2012['code'].isin(new_products_2012)].copy()
new_products_2017 = products_2017 - products_2012
new_products_2017_df = df_2017[df_2017['code'].isin(new_products_2017)].copy()
new_products_2022 = products_2022 - products_2017
new_products_2022_df = df_2022[df_2022['code'].isin(new_products_2022)].copy()

# Her yıla ait veri setine "Yıl" sütunu ekleyelim
new_products_2007_df['Yıl'] = 2007
new_products_2012_df['Yıl'] = 2012
new_products_2017_df['Yıl'] = 2017
new_products_2022_df['Yıl'] = 2022

# Üç yıldaki yeni ürünleri aynı data frame içine al
combined_new_products = pd.concat([new_products_2022_df, new_products_2017_df, new_products_2012_df, new_products_2007_df], ignore_index=True)

# 6 basamaklı HS kodlarını 10000'e bölerek 2 basamaklı sektörlerini bul
combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000

# Atlas verisini çağırıp 2 basamaklı HS kodlarının isimlerini çekelim
combined_atlas_df = pd.read_excel('0-) HS92 Sektör Kodları ve İsimleri.xlsx')
combined_atlas_df['2 Basamaklı HS Kodları'] = combined_atlas_df['2 Basamaklı HS Kodları'].str.extract(r'\((\d{2})')
combined_atlas_df['1 Basamaklı HS Kodları'] = combined_atlas_df['1 Basamaklı HS Kodları'].str.extract(r'\((\d{1})')

# Bir sözlük oluşturarak 2 basamaklı HS kodları ve isimleri arasında bir map oluşturalım
code_2_to_name_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['2 Basamaklı HS İsimleri']))
code_2_to_code_1_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS Kodları']))
code_1_to_name_mapping = dict(zip(combined_atlas_df['1 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS İsimleri']))

# 2 basamaklı HS kodlarını ve bunlara karşılık gelen 1 basamaklı kodları ve isimleri ekleyelim
combined_new_products['2 Basamaklı HS İsmi'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_name_mapping)
combined_new_products['1 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_code_1_mapping)
combined_new_products['1 Basamaklı HS İsmi'] = combined_new_products['1 Basamaklı HS Kodu'].map(code_1_to_name_mapping)

# 6 basamaklı HS kodunu ve detaylı sektör ismini zaten koruyoruz
combined_new_products = combined_new_products.rename(columns={'description': 'Detaylı Sektör İsmi', 'code': '6 Basamaklı HS Kodu'})  

# "Yıl" sütununu en başa taşıyalım
cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
combined_new_products = combined_new_products[cols]

# Excele kaydet
combined_new_products.to_excel("1-) Yeni ürünlerin sektörleri.xlsx")

In [3]:
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Tüm yılların verilerini tutacak liste
tum_yillar_verisi = []

# Her yılın verisini oku ve birleştir
for year in yillar:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)
    tum_yillar_verisi.append(df)

# Tüm yılları içeren DataFrame'i birleştir
birlesik_df = pd.concat(tum_yillar_verisi, ignore_index=True)

combined_atlas_df = pd.read_excel('0-) HS92 Sektör Kodları ve İsimleri.xlsx')

# 2 Basamaklı ve 1 Basamaklı kodları çıkar, temizle
combined_atlas_df['2 Basamaklı HS Kodları'] = combined_atlas_df['2 Basamaklı HS Kodları'].str.extract(r'\((\d{2})')
combined_atlas_df['1 Basamaklı HS Kodları'] = combined_atlas_df['1 Basamaklı HS Kodları'].str.extract(r'\((\d{1})')

# Sözlükler oluştur
code_2_to_name_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['2 Basamaklı HS İsimleri']))
code_2_to_code_1_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS Kodları']))
code_1_to_name_mapping = dict(zip(combined_atlas_df['1 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS İsimleri']))

# 6 basamaklı HS kodundan 2 basamaklı kodu hesapla (10000'e bölerek)
birlesik_df['k_2'] = birlesik_df['k'] // 10000

# Tek basamaklı HS kodları için sıfır ekle
birlesik_df['k_2'] = birlesik_df['k_2'].apply(lambda x: f'{int(x):02}')

# İsim eşlemeleri yap
birlesik_df['k_2_name'] = birlesik_df['k_2'].map(code_2_to_name_mapping)
birlesik_df['k_1'] = birlesik_df['k_2'].map(code_2_to_code_1_mapping)
birlesik_df['k_1_name'] = birlesik_df['k_1'].map(code_1_to_name_mapping)

# Sonuç DataFrame'i


In [7]:
# 2-) 1 Basamaklı sektörlerin ticaret hacimleri

import pandas as pd

# İlk olarak, her yıl ve k_1_name (sektör adı) bazında toplam değerleri bul
yearly_totals = birlesik_df.groupby(['t', 'k_1_name'])['v'].sum().reset_index(name='sector_total')

# Yıllık toplamı, her yıl için genel toplamı hesaplamak için kullan
yearly_totals['year_total'] = yearly_totals.groupby('t')['sector_total'].transform('sum')

# Her sektörün o yılki toplam içerisindeki oranını hesapla
yearly_totals['k_1_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
final_df = yearly_totals[['t', 'k_1_name', 'k_1_oran']]
final_df.columns = ['Yıl', '1 Basamaklı HS İsimleri', 'Ticaret Hacimleri']

# Excel dosyasına kaydet
final_df.to_excel("2-) 1 Basamaklı Sektörlerin Ticaret Hacimleri.xlsx", index=False)

In [8]:
# 2-) 2 Basamaklı sektörlerin ticaret hacimleri

import pandas as pd

# İlk olarak, her yıl ve k_2_name (sektör adı) bazında toplam değerleri bul
yearly_totals = birlesik_df.groupby(['t', 'k_2_name'])['v'].sum().reset_index(name='sector_total')

# Yıllık toplamı, her yıl için genel toplamı hesaplamak için kullan
yearly_totals['year_total'] = yearly_totals.groupby('t')['sector_total'].transform('sum')

# Her sektörün o yılki toplam içerisindeki oranını hesapla
yearly_totals['k_2_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
final_df = yearly_totals[['t', 'k_2_name', 'k_2_oran']]
final_df.columns = ['Yıl', '2 Basamaklı HS İsimleri', 'Ticaret Hacimleri']

# Excel dosyasına kaydet
final_df.to_excel("3-) 2 Basamaklı Sektörlerin Ticaret Hacimleri.xlsx", index=False)

In [9]:
birlesik_df["i"] ve birlesik_df[]

,t,i,j,k,v,q,k_2,k_2_name,k_1,k_1_name
0,2012,4,8,610469,2.734,0.044,61,"Apparel, knit (61 HS92)",0,Textiles (0 HS92)
1,2012,4,8,630900,0.110,0.082,63,Other made up textile articles (63 HS92),0,Textiles (0 HS92)
2,2012,4,12,91099,1.185,1.200,09,"Coffee, tea and spices (09 HS92)",1,Agriculture (1 HS92)
3,2012,4,32,392630,1.288,0.123,39,Plastics (39 HS92),5,Chemicals (5 HS92)
4,2012,4,32,710310,0.096,0.004,71,Precious metals and stones (71 HS92),2,Stone (2 HS92)
...,...,...,...,...,...,...,...,...,...,...
105197086,2021,894,858,240120,204.480,39.600,24,Tobacco (24 HS92),1,Agriculture (1 HS92)
105197087,2021,894,858,842123,0.997,0.040,84,Industrial Machinery (84 HS92),7,Machinery (7 HS92)
105197088,2021,894,858,842139,1.103,0.044,84,Industrial Machinery (84 HS92),7,Machinery (7 HS92)
105197089,2021,894,858,870810,0.086,0.003,87,Vehicles (87 HS92),6,Vehicles (6 HS92)


In [ ]:
# 4-) 1 Basamaklı sektörlerin ülkelerdeki ticaret hacimleri

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')

# Sektörlerin ülkelerdeki ticaret hacim oranları
yeni_urunler_ulke_bazli_ticaret_hacim_oranı = pd.DataFrame()
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]  
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2012-2016 için new_products_2012_df, 2017-2021 için new_products_2017_df 
        if year <= 2016:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
        else:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()
        if toplam_ticaret_hacmi > 0:  
            yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
        else:
            yeni_urunler_orani = 0 
yeni_urunler_ulke_bazli_ticaret_hacim_oranı.at[ulke, year] = yeni_urunler_orani
